In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [2]:
file = "gencode.v22.annotation.gtf.gz"

In [3]:
df = pd.read_csv(
        file, sep="\t", comment="#", compression='gzip', 
        usecols=[0, 1, 2, 3, 4, 8],
        header=None, names=(
            "chromosome_name", "annotation_source", "feature_type",
            "genomic_start_location", "genomic_end_location", 
            # "score", "genomic_strand", "genomic_phase", 
            "additional_info"
            )
    )
df

,chromosome_name,annotation_source,feature_type,genomic_start_location,genomic_end_location,additional_info
0,chr1,HAVANA,gene,11869,14409,"gene_id ""ENSG00000223972.5""; gene_type ""transc..."
1,chr1,HAVANA,transcript,11869,14409,"gene_id ""ENSG00000223972.5""; transcript_id ""EN..."
2,chr1,HAVANA,exon,11869,12227,"gene_id ""ENSG00000223972.5""; transcript_id ""EN..."
3,chr1,HAVANA,exon,12613,12721,"gene_id ""ENSG00000223972.5""; transcript_id ""EN..."
4,chr1,HAVANA,exon,13221,14409,"gene_id ""ENSG00000223972.5""; transcript_id ""EN..."
...,...,...,...,...,...,...
2563666,chrM,ENSEMBL,transcript,15888,15953,"gene_id ""ENSG00000210195.2""; transcript_id ""EN..."
2563667,chrM,ENSEMBL,exon,15888,15953,"gene_id ""ENSG00000210195.2""; transcript_id ""EN..."
2563668,chrM,ENSEMBL,gene,15956,16023,"gene_id ""ENSG00000210196.2""; gene_type ""Mt_tRN..."
2563669,chrM,ENSEMBL,transcript,15956,16023,"gene_id ""ENSG00000210196.2""; transcript_id ""EN..."


In [4]:
def parseinfo(s):
    
    items = s['additional_info'].replace(" ", "").split(";")
    gene_id = None
    gene_type = None
    gene_name = None
    for i in items:
        if 'gene_id' in i:
            gene_id = i.split('"')[1]
        elif 'gene_type' in i:
            gene_type = i.split('"')[1]
        elif 'gene_name' in i:
            gene_name = i.split('"')[1]
    return gene_id, gene_type, gene_name

df[['ensembl_id', 'gene_type', 'gene_name']] = df[['additional_info']].progress_apply(parseinfo, axis=1, result_type='expand')

100%|█████████████████████████████████████████████████████████████████████| 2563671/2563671 [02:13<00:00, 19139.68it/s]


In [5]:
new_df = df[["chromosome_name", "annotation_source", "feature_type",
            "genomic_start_location", "genomic_end_location", 
            # "score", "genomic_strand", "genomic_phase", 
            "ensembl_id", "gene_type", "gene_name"]]
new_df

,chromosome_name,annotation_source,feature_type,genomic_start_location,genomic_end_location,ensembl_id,gene_type,gene_name
0,chr1,HAVANA,gene,11869,14409,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1
1,chr1,HAVANA,transcript,11869,14409,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1
2,chr1,HAVANA,exon,11869,12227,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1
3,chr1,HAVANA,exon,12613,12721,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1
4,chr1,HAVANA,exon,13221,14409,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1
...,...,...,...,...,...,...,...,...
2563666,chrM,ENSEMBL,transcript,15888,15953,ENSG00000210195.2,Mt_tRNA,MT-TT
2563667,chrM,ENSEMBL,exon,15888,15953,ENSG00000210195.2,Mt_tRNA,MT-TT
2563668,chrM,ENSEMBL,gene,15956,16023,ENSG00000210196.2,Mt_tRNA,MT-TP
2563669,chrM,ENSEMBL,transcript,15956,16023,ENSG00000210196.2,Mt_tRNA,MT-TP


In [6]:
def unioninterval(l):
    
    def get_union(a, b):
        a0, a1 = a
        b0, b1 = b
        if a1 + 1 == b0:
            return [1, a0, b1]
        if a1 < b0:
            return [2, a, b]
        if a0 <= b0:
            return [1, a0, max(a1, b1)]
    
    l.sort(key=lambda x: x[0])
    i, j = 0, len(l) - 1
    while True:
        if i == j:
            break
        rangea, rangeb = l.pop(i), l.pop(i)
        union_i = get_union(rangea, rangeb)
        if union_i[0] == 1:
            l.insert(i, [union_i[1], union_i[2]])
        elif union_i[0] == 2:
            l.insert(i, union_i[1])
            l.insert(i + 1, union_i[2])
            i += 1
        j = len(l) - 1
    return l

def addup(sdf):
    interval = unioninterval(
        sdf[['genomic_start_location', 'genomic_end_location']].values.tolist()
    )
    length = 0
    for x in interval:
        length += x[1] - x[0] + 1        
    return pd.Series([sdf.iloc[0, :]['gene_name'], length])

exon_df = new_df.loc[new_df['feature_type'] == 'exon', ].groupby('ensembl_id').progress_apply(addup)
exon_df.columns = ['symbol', 'length']
exon_df

100%|██████████████████████████████████████████████████████████████████████████| 60483/60483 [00:43<00:00, 1403.04it/s]


,symbol,length
ensembl_id,,
ENSG00000000003.13,TSPAN6,4535
ENSG00000000005.5,TNMD,1610
ENSG00000000419.11,DPM1,1207
ENSG00000000457.12,SCYL3,6883
ENSG00000000460.15,C1orf112,5967
...,...,...
ENSGR0000275287.3,Metazoa_SRP,290
ENSGR0000276543.3,AJ271736.1,68
ENSGR0000277120.3,MIR6089,64


In [7]:
exon_df.to_csv('gene.v22.simple.csv')